In [1]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


In [2]:
import pandas as pd
from pyomo.environ import NonNegativeReals, NonNegativeIntegers
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Set, Param, NonNegativeReals, minimize

Read the parameter data:

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv('optimizer/optimization_data/parameters.csv')

# Choose the 24-hour period you want to optimize (for example, starting on 2024-07-01 00:00:00)
start_date = '2023-06-01 00:00:00'
end_date = '2023-06-02 00:00:00'

# Filter the data for the chosen 24-hour period
data = df[(df['Time'] >= start_date) & (df['Time'] < end_date)]

# Ensure that the data is sorted by timestamp and reset the index
data = data.sort_values(by='Time').reset_index(drop=True)

# Check the filtered data
data


,Time,production_schedule,compressor_consumption,operational_presence,testing_schedule,active_computers,maintenance_status,lightbulbs_active,workload,volume_production_waste,fabric_in_chamber,active_wall_plugs,number_of_workers,heat_index,price_mWh,Hour,efficiency_adjusted_power,technological_centers_consumption
0,2023-06-01 00:00:00,0,1.008715,0.000000,0.091031,0,0,0,0.000000,0,0.000000,0,0,14.27,94.09,0,0.608466,1.001112
1,2023-06-01 01:00:00,31,16.339983,4.370405,15.624779,1,15,1,23.990761,17,11.078917,2,0,13.77,87.00,1,0.609133,30.909301
2,2023-06-01 02:00:00,0,16.491206,3.889101,17.954943,1,24,1,25.457470,15,3.723168,0,73,13.60,87.00,2,0.609876,28.756900
3,2023-06-01 03:00:00,0,16.465381,4.362756,11.717233,1,1,2,19.707243,11,5.162315,1,0,13.77,89.01,3,0.610317,28.556674
4,2023-06-01 04:00:00,21,17.650720,5.727740,19.267657,2,34,3,22.953785,13,9.049066,2,0,14.27,81.00,4,0.609933,28.556675
5,2023-06-01 05:00:00,12,17.272565,3.986613,6.058799,0,0,2,19.074010,9,7.968235,2,0,15.06,80.10,5,0.609537,28.381483
6,2023-06-01 06:00:00,12,27.863739,3.245102,21.827980,2,45,1,17.939803,13,5.510599,1,0,16.10,88.10,6,0.609557,29.707961
7,2023-06-01 07:00:00,41,52.271777,5.813971,14.187327,1,24,3,21.181540,18,9.834054,3,0,17.31,98.50,7,0.609516,27.355346
8,2023-06-01 08:00:00,31,55.474211,4.728541,18.233220,1,44,2,18.413512,17,7.561134,2,0,18.60,99.07,8,0.609570,28.356455
9,2023-06-01 09:00:00,32,54.667023,3.414813,15.624900,2,35,2,21.713372,24,7.653581,3,29,19.89,87.30,9,0.609632,28.857011


Create a model:

In [4]:
# Define the model
model = ConcreteModel(name="Daily_Price_Optimizer")

# Display the model
print(model)

Daily_Price_Optimizer


**Define Sets:** 

In [5]:
# Define the set of hours
model.hours = Set(initialize=range(0, 24))

**Define Parameters:**
Parameters are values that are given to us, and we don’t have control over them.

In [6]:
# Define the parameters and initialize them with data from the CSV
model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['production_schedule'].to_dict())

model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['maintenance_status'].to_dict())

model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['volume_production_waste'].to_dict())

model.number_of_workers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['number_of_workers'].to_dict())

model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['heat_index'].to_dict())

model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['efficiency_adjusted_power'].to_dict())

model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['compressor_consumption'].to_dict())

model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['operational_presence'].to_dict())

model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['fabric_in_chamber'].to_dict())

model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['testing_schedule'].to_dict())

model.workload = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['workload'].to_dict())

model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['technological_centers_consumption'].to_dict())

model.lightbulbs_active = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['lightbulbs_active'].to_dict())

model.active_wall_plugs = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_wall_plugs'].to_dict())

model.active_computers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_computers'].to_dict())

model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['price_mWh'].to_dict())

# Display the model
print(model)

Daily_Price_Optimizer


**Define Decision Variables with bounds:**
This means  x  and  y  are the variables we want to determine, and they must be non-negative (can’t be less than zero).

In [7]:


# Define the decision variables
model.power_transport_vehicles = Var(model.hours, within=NonNegativeReals, bounds=(0, 200))
model.set_point = Var(model.hours, within=NonNegativeIntegers, bounds=(18, 25))
model.num_active_chiller = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 10))
model.standby_power_down = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 1))
model.active_printers = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.active_coffee_machines = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.num_servers = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_non_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_hdds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_ssds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))

# Define additional decision variables for constraint outcomes
model.chiller_energy = Var(model.hours, within=NonNegativeReals)
model.data_center_energy = Var(model.hours, within=NonNegativeReals)
model.production_energy = Var(model.hours, within=NonNegativeReals)
model.uta_energy = Var(model.hours, within=NonNegativeReals)
model.office_energy = Var(model.hours, within=NonNegativeReals)

# Display the model to check the variables
print(model)

Daily_Price_Optimizer


**Define the constraints:**

In [8]:
import joblib    

chiller_model = joblib.load('equation_modeling/models/chiller_consumption_model.joblib')
data_center_model = joblib.load('equation_modeling/models/data_center_consumption_model.joblib')
office_model = joblib.load('equation_modeling/models/office_consumption_model.joblib')
production_model = joblib.load('equation_modeling/models/production_consumption_model.joblib')
uta_model = joblib.load('equation_modeling/models/uta_consumption_model.joblib')

In [9]:
chiller_intercept = chiller_model['intercept']
chiller_coefficients = chiller_model['coefficients']
chiller_features = chiller_model['features']

# Define the features as Pyomo expressions
def heat_index_set_point_diff_expr(model, h):
    return model.heat_index[h] - model.set_point[h]

# Define the chiller energy consumption constraint
def chiller_energy_constraint_rule(model, h):
    # Calculate the adjusted energy using the linear regression model
    COP_value = 3.0 
    
    adjusted_energy = ((
        chiller_intercept +
        chiller_coefficients[0] * heat_index_set_point_diff_expr(model, h) +
        chiller_coefficients[1] * model.efficiency_adjusted_power[h] +
        chiller_coefficients[2] * model.num_active_chiller[h]
    ) / (COP_value * model.efficiency_adjusted_power[h]))
    
    return model.chiller_energy[h] == adjusted_energy

# Add the constraint to the model
model.chiller_energy_constraint = Constraint(model.hours, rule=chiller_energy_constraint_rule)

In [10]:
data_center_coefficients = data_center_model['coefficients']
data_center_features = data_center_model['features']

# Define the chiller energy consumption constraint
def data_center_constraint_rule(model, h):
    
    energy = (
        data_center_coefficients['server'] * model.num_servers[h] +
        data_center_coefficients['network_switch_poe'] * model.num_network_switches_poe[h] +
        data_center_coefficients['network_switch_non_poe'] * model.num_network_switches_non_poe[h] +
        data_center_coefficients['hdd'] * model.num_hdds[h] +
        data_center_coefficients['ssd'] * model.num_ssds[h]
    ) 
    
    return model.data_center_energy[h] == energy


# Add the constraint to the model
model.data_center_energy_constraint = Constraint(model.hours, rule=data_center_constraint_rule)

In [11]:
production_intercept = production_model['intercept']
production_coefficients = production_model['coefficients']
production_features = production_model['features']

# Define the chiller energy consumption constraint
def production_energy_constraint_rule(model, h):
    
    energy = (
        production_intercept +
        production_coefficients[0] * model.power_transport_vehicles[h] +
        production_coefficients[1] * model.production_schedule[h] +
        production_coefficients[2] * model.maintenance_status[h] +
        production_coefficients[3] * model.volume_production_waste[h] +
        production_coefficients[4] * model.number_of_workers[h]
    ) 
    
    return model.production_energy[h] == energy

# Add the constraint to the model
model.production_energy_constraint = Constraint(model.hours, rule=production_energy_constraint_rule)

In [12]:
uta_intercept = uta_model['intercept']
uta_coefficients = uta_model['coefficients']
uta_features = uta_model['features']

# Define the chiller energy consumption constraint
def uta_energy_constraint_rule(model, h):
    
    energy = (
        uta_intercept +
        uta_coefficients[0] * model.operational_presence[h] +
        uta_coefficients[1] * model.fabric_in_chamber[h] +
        uta_coefficients[2] * model.testing_schedule[h] +
        uta_coefficients[3] * model.workload[h] +
        uta_coefficients[4] * model.standby_power_down[h]
    ) 
    
    return model.uta_energy[h] == energy

# Add the constraint to the model
model.uta_energy_constraint = Constraint(model.hours, rule=uta_energy_constraint_rule)

In [13]:
office_intercept = office_model['intercept']
office_coefficients = office_model['coefficients']
office_features = office_model['features']

# Define the chiller energy consumption constraint
def office_energy_constraint_rule(model, h):
    
    energy = (
        office_intercept +
        office_coefficients[0] * model.lightbulbs_active[h] +
        office_coefficients[1] * model.active_wall_plugs[h] +
        office_coefficients[2] * model.active_computers[h] +
        office_coefficients[3] * model.active_printers[h] +
        office_coefficients[4] * model.active_coffee_machines[h]
    ) 
    
    return model.office_energy[h] == energy

# Add the constraint to the model
model.office_energy_constraint = Constraint(model.hours, rule=office_energy_constraint_rule)

**Define Objective Function:**
This means we want to minimize the total cost

In [14]:


# Define the objective function
def total_cost_rule(model):
    return sum(
        (model.price_mWh[h] / 1000) * (
            model.compressors_energy[h] +
            model.technological_centers_energy[h] +
            model.chiller_energy[h] +
            model.data_center_energy[h] +
            model.production_energy[h] +
            model.uta_energy[h] +
            model.office_energy[h]
        ) for h in model.hours
    )

# Add the objective function to the model
model.total_cost = Objective(rule=total_cost_rule, sense=minimize)

**Solve the optimization problem:**

In [15]:
# Solve the model using GLPK
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Check the solver status
if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
    print("Solver found an optimal solution.")
else:
    print("Solver did not find an optimal solution. Status: ", results.solver.status)

# Display the objective function value
print("Total Cost: ", model.total_cost())

# Display the values of the decision variables
for h in model.hours:
    print(f"Hour {h}:")
    print(f"  Power Transport Vehicles: {model.power_transport_vehicles[h].value}")
    print(f"  Set Point: {model.set_point[h].value}")
    print(f"  Number of Active Chillers: {model.num_active_chiller[h].value}")
    print(f"  Standby Power Down: {model.standby_power_down[h].value}")
    print(f"  Active Printers: {model.active_printers[h].value}")
    print(f"  Active Coffee Machines: {model.active_coffee_machines[h].value}")
    print(f"  Number of Servers: {model.num_servers[h].value}")
    print(f"  Number of Network Switches (PoE): {model.num_network_switches_poe[h].value}")
    print(f"  Number of Network Switches (Non-PoE): {model.num_network_switches_non_poe[h].value}")
    print(f"  Number of HDDs: {model.num_hdds[h].value}")
    print(f"  Number of SSDs: {model.num_ssds[h].value}")
    print(f"  Chiller Energy: {model.chiller_energy[h].value}")
    print(f"  Data Center Energy: {model.data_center_energy[h].value}")
    print(f"  Production Energy: {model.production_energy[h].value}")
    print(f"  UTA Energy: {model.uta_energy[h].value}")
    print(f"  Office Energy: {model.office_energy[h].value}")
    print(f"  Compressors Energy: {model.compressors_energy[h]}")
    print(f"  Technological Centers Energy: {model.technological_centers_energy[h]}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmps86cejo8.glpk.raw
 --wglp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpukk3w__4.glpk.glp
 --cpxlp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpjkivh45b.pyomo.lp
Reading problem data from '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpjkivh45b.pyomo.lp'...
120 rows, 385 columns, 384 non-zeros
240 integer variables, 24 of which are binary
1500 lines were read
Writing problem data to '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpukk3w__4.glpk.glp'...
1470 lines were written
GLPK Integer Optimizer 5.0
120 rows, 385 columns, 384 non-zeros
240 integer variables, 24 of which are binary
Preprocessing...
11 rows, 22 columns, 22 non-zeros
22 integer variables, none of which are binary
Scaling...
 A: min|aij| =  8.255e-01  max|aij| =  1.224e+01  ratio =  1.483e+01
GM: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
EQ: 

# DO IT ALL

In [16]:
data = pd.read_csv('optimizer/optimization_data/parameters.csv')
data['Time'] = pd.to_datetime(data['Time'])

In [17]:
from datetime import timedelta

# Create empty dataframes to store the results
hourly_results_df = pd.DataFrame()
daily_results_df = pd.DataFrame()

# Define the start and end dates for the iteration
start_date = pd.to_datetime('2023-06-01')
end_date = pd.to_datetime('2023-07-01')

# Iterate over each day
current_date = start_date

while current_date < end_date:
    # Define the 24-hour period
    start_datetime = current_date
    end_datetime = current_date + timedelta(days=1)

    # Filter the data for the current 24-hour period
    filtered_data = data[(data['Time'] >= start_datetime) & (data['Time'] < end_datetime)]
    filtered_data = filtered_data.sort_values(by='Time').reset_index(drop=True)

    # Initialize the model parameters with filtered data
    model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['production_schedule'].to_dict())
    model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['maintenance_status'].to_dict())
    model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['volume_production_waste'].to_dict())
    model.number_of_workers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['number_of_workers'].to_dict())
    model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['heat_index'].to_dict())
    model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['efficiency_adjusted_power'].to_dict())
    model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['compressor_consumption'].to_dict())
    model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['operational_presence'].to_dict())
    model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['fabric_in_chamber'].to_dict())
    model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['testing_schedule'].to_dict())
    model.workload = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['workload'].to_dict())
    model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['technological_centers_consumption'].to_dict())
    model.lightbulbs_active = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['lightbulbs_active'].to_dict())
    model.active_wall_plugs = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_wall_plugs'].to_dict())
    model.active_computers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_computers'].to_dict())
    model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['price_mWh'].to_dict())

    # Solve the model
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=True)

    # Check the solver status
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print(f"Solver found an optimal solution for {current_date.date()}.")
    else:
        print(f"Solver did not find an optimal solution for {current_date.date()}. Status: {results.solver.status}")

    # Store hourly results
    for h in model.hours:
        hourly_results_df = hourly_results_df.concat({
            'Date': current_date,
            'Hour': h,
            'Power Transport Vehicles': model.power_transport_vehicles[h].value,
            'Set Point': model.set_point[h].value,
            'Number of Active Chillers': model.num_active_chiller[h].value,
            'Standby Power Down': model.standby_power_down[h].value,
            'Active Printers': model.active_printers[h].value,
            'Active Coffee Machines': model.active_coffee_machines[h].value,
            'Number of Servers': model.num_servers[h].value,
            'Number of Network Switches (PoE)': model.num_network_switches_poe[h].value,
            'Number of Network Switches (Non-PoE)': model.num_network_switches_non_poe[h].value,
            'Number of HDDs': model.num_hdds[h].value,
            'Number of SSDs': model.num_ssds[h].value,
            'Compressor Energy': model.compressors_energy[h],
            'Technological Centers Energy': model.technological_centers_energy[h],
            'Chiller Energy': model.chiller_energy[h].value,
            'Data Center Energy': model.data_center_energy[h].value,
            'Production Energy': model.production_energy[h].value,
            'UTA Energy': model.uta_energy[h].value,
            'Office Energy': model.office_energy[h].value,
            'Total Cost': model.total_cost()
        }, ignore_index=True)

    # Store daily results
    daily_results_df = daily_results_df.concat({
        'Date': current_date,
        'Total Cost': model.total_cost()
    }, ignore_index=True)

    # Move to the next day
    current_date += timedelta(days=1)

# Reset index of the dataframes
hourly_results_df.reset_index(drop=True, inplace=True)
daily_results_df.reset_index(drop=True, inplace=True)

# Save the results to CSV files
hourly_results_df.to_csv('hourly_results.csv', index=False)
daily_results_df.to_csv('daily_results.csv', index=False)

(type=<class 'pyomo.core.base.param.IndexedParam'>) on block
Daily_Price_Optimizer with a new Component (type=<class
'pyomo.core.base.param.IndexedParam'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.IndexedParam'>) on block
Daily_Price_Optimizer with a new Component (type=<class
'pyomo.core.base.param.IndexedParam'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.IndexedParam'>) on block
Daily_Price_Optimizer with a new Component (type=<class
'pyomo.core.base.param.IndexedParam'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.IndexedParam'>) on block
Daily_Price_Optimizer with a new Component (type=<class
'pyomo.core.base.param.IndexedParam'>

AttributeError: 'DataFrame' object has no attribute 'concat'